# TITANIC MACJINE LEARNING FROM DISASTER

## 1. a)Defining the Question

> We intend to predict the survival rate of passengers of the titanic.

## b) Defining the metric of success

1. Accurate determination of factors that greatly affected passenger survival.
2. Attaining prediction accuracies greater than 80%.


## c) Understanding the context

> The Titanic sunk in 1912, when data wa recorded on paper and stored in files. Furthermore, some of the data was estimated, as data recording was not as detailed as it is today. Therefore, obtaining an accuracy greater than 80% after model otpimization will be satisfactory.



## d) Recording the experimental design

1. Data sourcing/loading
2. Data Understanding
3. Data Relevance
4. External Dataset Validation
5. Data Preparation
6. Univariate Analysis
7. Bivariate Analysis
8. Multivariate Analysis
9. Implementing the solution
10. Challenging the solution
11. Conclusion
12. Follow up questions

## e) Data relevance

1. The data should have variables that adequately contribute to predicting passenger survival.

2. The dataset should lead to a high model fit (high accuracy, after all possible model optimization procedures have been applied.

We shall be using the datasets below:

i. Credit test dataset ( https://www.kaggle.com/competitions/titanic/data?select=train.csv#:~:text=calendar_view_week-,train,-.csv )

ii. Credit train dataset ( https://www.kaggle.com/competitions/titanic/data?select=test.csv#:~:text=test.-,csv,-calendar_view_week )

# 2. Data Understanding

In [1]:
# lets import the libraries we need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, f1_score, precision_score, recall_score, classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB, CategoricalNB
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image
import pydotplus
import os
import pywedge as pw

# filter to ignore warnings
import warnings
warnings.filterwarnings('ignore')

### a) reading the data

In [2]:
# lets read the dataset into a dataframe
train = pd.read_csv(
    'urbanization-external-data/Percentage_Distribution_of_Households_by_Age_and_Sex_of_Household_Head.csv')


In [3]:
train.head(5)

,County,Gender,Age_Group_of_Household_head_15-24_%_,Age_Group_of_Household_head_25-34_%,Age_Group_of_Household_head_35-59_%,Age_Group_of_Household_head_60+_%,Number_of_households,County_Centroid,OBJECTID
0,Mombasa,Female,16.8%,36.8%,39.5%,6.8%,59858,"(-4.018795613, 39.653975520000003)",0
1,Kwale,Female,10.9%,25.4%,46.5%,17.1%,40265,"(-4.1398571640000004, 39.15036963)",1
2,Kilifi,Female,12.5%,26.6%,46.2%,14.7%,64963,"(-3.1731002359999998, 39.683055940000003)",2
3,Tana Rriver,Female,16.9%,25.6%,43.3%,14.2%,14447,"(-1.557428783, 39.412916930000002)",3
4,Lamu,Female,9.5%,22.6%,48.7%,19.2%,5515,"(-2.0391399720000001, 40.767880740000003)",4


## Instantiating the Pywedge_Charts Class

For inputs, pywedge takes:

1. Dataframe
2. c = any redundant column to be removed
3. y = target column as a string

In [4]:
mc = pw.Pywedge_Charts(train, c = None, y = 'County')

In [ ]:
charts = mc.make_charts()

### b) Checking the data

Number of Records

In [13]:
# lets print the number of rows and columns for train and test sets
print(f'The train data has {train.shape[0]} rows and {train.shape[1]} columns')
print(f'The test data has {test.shape[0]} rows and {test.shape[1]} columns')

The train data has 891 rows and 12 columns
The test data has 418 rows and 11 columns


> The train data has more columns than the test data by one namely the "Survived" column.

Top dataset view

In [14]:
# lets view the top of the datasets
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
# preview of the test column
test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


> The test data does not have the label column

In [16]:
# lets see the bottom records
train.tail(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [17]:
# for the test
test.tail(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


Checking the data types

In [18]:
# let's see the data types of the columns for train
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


> The data types are okay only that the age, embarked and cabin have too many missing values.

In [19]:
# for test
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


> Age, fare, cabin seem to have missing values, otherwise everything is okay.

# 3. External dataset validation

The dataset is similar to a Kaggle Titanic dataset. It has also been used by the Computer Science Department in Stanford University as part of course work. Stanford is a well known ivy league university, therefore, the dataset is valid.



# 4. Data Preparation

### a) Consistency

In [29]:
# test
test.duplicated().any()

False

In [30]:
# train
train.duplicated().any()

False

> We dont have duplicates in the dataset

### b) Validation

> We are dropping the passenger id column because we can use passenger names for identification

> We also check the relevance of the ticket column

In [34]:
# lets check the relevance of the ticket column
print(f'The percentage of unique values in the train dataset is {round((len(train.Ticket.unique()) / train.shape[0]) * 100)} %')

print(f'The percentage of unique values in the test dataset is {round((len(test.Ticket.unique()) / test.shape[0]) * 100)} %')

The percentage of unique values in the train dataset is 76 %
The percentage of unique values in the test dataset is 87 %


> The tickets have high numbers of unique values which can be used to identify the individual survivors

In [35]:
# lets drop the passenger id column
train.drop(['PassengerId'], axis = 1, inplace= True)
test.drop(['PassengerId'], axis = 1, inplace = True)


In [36]:
# lets check to see if changes have been made
print(f'The train has {len(train.columns.values)}')
print(f'The test has {len(test.columns.values)}')

The train has 11
The test has 10


### c) Completeness

> We are checking for the percentage of missing values

In [39]:
# let's create a function to find the percentage of missing values
def PercentageMissing(data):
   for col in data.columns.to_list():
      missing = data[col].isnull().sum()
      if missing > 0:
         print(f'{col} = {(missing/data.shape[0]) * 100 }%')

In [40]:
# checking for missing values in the train
PercentageMissing(train)

Age = 19.865319865319865%
Cabin = 77.10437710437711%
Embarked = 0.22446689113355783%


In [41]:
# for test
PercentageMissing(test)

Age = 20.574162679425836%
Fare = 0.23923444976076555%
Cabin = 78.22966507177034%


> We are dropping the cabin column because it has more than 75 percent missing in both dataframes.

In [42]:
# dropping cabin
train.drop(['Cabin'], axis = 1, inplace= True)
test.drop(['Cabin'], axis= 1, inplace= True)

In [43]:
# lets see the unique passenger class values
train.Pclass.unique()

array([3, 1, 2])

In [ ]:
# lets create a function to fill in the missing values with the column mean or the highest column frequency

